In [3]:
import re
import string
import numpy as np
import pandas as pd
import pickle
import nltk
nltk.download('popular', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import clear_output

def preprocess_text(text):
    text = text.lower()  # Приведение текста к нижнему регистру
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)  # Удаление markdown-ссылок
    text = re.sub(r'@\w+', '', text)  # Удаление упоминаний (handle)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Удаление ссылок
    text = re.sub(r'<.*?>+', '', text)  # Удаление HTML-тегов
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # Удаление знаков препинания
    text = re.sub(r'\n', '', text)  # Удаление символов новой строки
    text = re.sub(r'\w*\d\w*', '', text)  # Удаление слов с цифрами
    return text.strip()  # Удаление лишних пробелов в начале и конце текста

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize(tokens):
    pos_tagged = nltk.pos_tag(tokens) 
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else: 
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return ' '.join(lemmatized_sentence)

model = pickle.load(open('model.pkl', 'rb'))
statement = input()
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = lemmatize(word_tokenize(statement))
vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))
tfidf_tokens = vectorizer.transform([lemmatized_tokens])
additional_features = np.array([len(statement), len([i for i in lemmatized_tokens if i in ['i','ive','im','i`m','i`ve']])])
X = hstack([tfidf_tokens, additional_features])
labels = ['Anxiety', 'Bipolar', 'Depression', 'Normal', 'Personality disorder', 'Stress', 'Suicidal']
print('Предсказанный диагноз - ' + labels[model.predict(X)[0]])

Простите, сэр, я ищу работу. Собственно, я хочу найти работу, на которой мог бы чему-то научиться и вырасти. Я труженик, у меня высокие цели, и, как вы убедились, я настойчив. Я не обманываю себя. Будучи воспитан с чувством высокого самоуважения, столь популярного в школах, я полагаю, что мои потребности будут учтены. Но понял, что современная рабочая культура более не заинтересована в преданности рабочему классу, как это было у предыдущих поколений. Я верю, что удача приходит к тем, сэр, кто готов оторвать задницу от дивана. К таким как вы, кто сам достиг вершины горы, не падая вниз. Мой девиз если хотите выиграть в лотерею, заработайте на билет. Я упомянул, что работал в автомастерской? Что скажете? Могу начать завтра, или даже сегодня?
Предсказанный диагноз - Normal
